In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!git clone https://github.com/utkarsh512/CreateDebate-Scraper.git

In [ ]:
%cd CreateDebate-Scraper/src/nested/

In [ ]:
import re
import pickle
from thread import Thread, Comment
import numpy as np
import pandas as pd

In [ ]:
dir = '/content/gdrive/MyDrive/DL/CreateDebate/politics2/{}.log'
comments_with_score = list()

with open(dir.format('comments_with_score'), 'rb') as f:
    comments_with_score = pickle.load(f)

In [ ]:
comments_with_score[0]

In [ ]:
TOPIC = 'politics2'

In [ ]:
reader_addr = f'/content/gdrive/MyDrive/DL/CreateDebate/{TOPIC}/threads.log'
reader = open(reader_addr, 'rb')
threads = list()
e = Thread()
try:
    while True:
        e = pickle.load(reader)
        threads.append(e)
except:
    reader.close()

In [ ]:
comments = list()
times = list()
scores = list()

for i in range(len(comments_with_score)):
    scores.append(comments_with_score[i][0])

for thread in threads:
    for key in thread.comments.keys():
        comment = thread.comments[key]
        cur_text = comment.body
        cur_time = comment.time 
        comments.append(cur_text)
        times.append(cur_time)

In [ ]:
for i in range(len(comments)):
    comments[i] = re.sub("\s+", " ", comments[i])

In [ ]:
print(len(scores), len(times))

In [ ]:
min_year = 2022
max_year = 0
for i in range(len(times)):
    x = times[i]
    if x == 'Not Available':
        continue
    x = int(x.split('-')[0])
    min_year = min(min_year, x)
    max_year = max(max_year, x)
print(min_year) 

In [ ]:
comments_by_year = dict()

In [ ]:
for i in range(min_year, max_year + 1):
    ys = str(i)
    comments_by_year[ys] = dict()
    for j in range(1, 13):
        ms = str(j)
        if len(ms) == 1:
            ms = '0' + ms
        comments_by_year[ys][ms] = list()

In [ ]:
cnt = 0

for i in range(len(times)):
    if times[i] == 'Not Available':
        continue
    cnt += 1
    x = times[i].split('-')
    comments_by_year[x[0]][x[1]].append((comments[i], scores[i]))

print(cnt)

In [ ]:
ad_hominem = dict()
none = dict()

In [ ]:
for i in range(min_year, max_year + 1):
    ys = str(i)
    for k, v in comments_by_year[ys].items():
        for x in v:
            if x[1] < 0.5:
                try:
                    ad_hominem[ys].append(x[0])
                except:
                    ad_hominem[ys] = list()
                    ad_hominem[ys].append(x[0])
            else:
                try:
                    none[ys].append(x[0])
                except:
                    none[ys] = list()
                    none[ys].append(x[0])

In [ ]:
for i in range(min_year, max_year + 1):
    with open(f'/content/gdrive/MyDrive/DL/CreateDebate/{TOPIC}/temporal/{i}_ah.txt', 'w', encoding='utf-8') as f:
        for x in ad_hominem[str(i)]:
            f.write(x + '\n')

In [ ]:
for i in range(min_year, max_year + 1):
    with open(f'/content/gdrive/MyDrive/DL/CreateDebate/{TOPIC}/temporal/{i}_none.txt', 'w', encoding='utf-8') as f:
        for x in none[str(i)]:
            f.write(x + '\n')

In [ ]:
for i in range(min_year, max_year + 1):
    with open(f'/content/gdrive/MyDrive/DL/CreateDebate/{TOPIC}/temporal/{i}_all.txt', 'w', encoding='utf-8') as f:
        for x in none[str(i)]:
            f.write(x + '\n')
        for x in ad_hominem[str(i)]:
            f.write(x + '\n')